In [1]:
# Esta version es la que funciona mejor ya que ademas de preparar el dataset visualiza el resultado OJO OJO OJO
# Incluye la version definitiva incluyendo la funcion de visualizacion
# Ademas aplica un nivel razonable y seguro de DA ... hemos excluido las transformaciones Geométricas por considerar
# que pueden tener efecto distorsionante sobre el dataset (rotaciones, escalados, etc.)
# Se ha añadido la posibilidad de visualizar el dataset generado

In [2]:
# --- Importar las librerías necesarias ---

import torch
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
from sklearn.model_selection import train_test_split

# PASO 1: Definición de la clase BloodCellDataset
# Esta clase debe manejar la carga de imágenes y anotaciones, así como las transformaciones necesarias.
# Modificada para que filtre las bounding boxes degeneradas o inválidas.

class BloodCellDataset(Dataset):
    def __init__(self, data_root, annotations_df, image_size=(416, 416), transform=None):
        self.data_root = data_root
        self.image_folder = os.path.join(data_root, 'BCCD')
        self.image_size = image_size
        self.transform = transform
        
        self.class_name_to_id = {
            'RBC': 0, 'WBC': 1, 'Platelets': 2
        }
        self.class_id_to_name = {
            0: 'RBC', 1: 'WBC', 2: 'Platelets'
        }
        
        self.image_annotations = {}
        # Filtrar el DataFrame de anotaciones para eliminar filas con valores NaN en columnas clave
        annotations_df = annotations_df.dropna(subset=['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'cell_type'])
        
        for filename, group in annotations_df.groupby('filename'):
            bboxes_pixel_list = []
            for idx, row in group.iterrows():
                cell_type = str(row['cell_type']) # Asegurarse de que sea string
                xmin = int(row['xmin'])
                xmax = int(row['xmax'])
                ymin = int(row['ymin'])
                ymax = int(row['ymax']) 
                
                class_id = self.class_name_to_id.get(cell_type)
                if class_id is None:
                    print(f"Advertencia: Tipo de célula desconocido '{cell_type}' en el archivo {filename}. Saltando anotación.")
                    continue

                # Asegurarse de que xmin < xmax y ymin < ymax antes de guardar
                if xmin >= xmax or ymin >= ymax:
                    # print(f"Advertencia: Bounding box degenerado o inválido en {filename}: ({xmin}, {ymin}, {xmax}, {ymax}). Saltando.")
                    continue # Saltar esta bbox inválida

                bboxes_pixel_list.append([xmin, ymin, xmax, ymax, class_id])
            
            # Solo añadir la imagen si tiene al menos una bbox válida
            if bboxes_pixel_list:
                self.image_annotations[filename] = bboxes_pixel_list
        
        self.image_files = list(self.image_annotations.keys())
        print(f"Dataset inicializado con {len(self.image_files)} imágenes.")
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        original_h, original_w, _ = image.shape
        print(f"DEBUG: Imagen original (H, W): ({original_h}, {original_w})")

        bboxes_pixel = self.image_annotations.get(img_name, [])
        print(f"DEBUG: __getitem__ para {img_name}. Bboxes iniciales (píxeles): {len(bboxes_pixel)}")
        if bboxes_pixel:
            print(f"DEBUG: Primer bbox pixel: {bboxes_pixel[0]}")
        
        # --- NORMALIZAR BBOXES A [0, 1] ANTES DE ALBUMENTATIONS ---
        # Albumentations espera bboxes normalizadas si format='albumentations'
        bboxes_normalized_initial = []
        class_labels = [] # class_labels se mantiene
        for bbox_px in bboxes_pixel:
            xmin_px, ymin_px, xmax_px, ymax_px, class_id = bbox_px
            
            # Normalizar las coordenadas a [0, 1] usando las dimensiones originales
            xmin_norm = xmin_px / original_w
            ymin_norm = ymin_px / original_h
            xmax_norm = xmax_px / original_w
            ymax_norm = ymax_px / original_h
            
            bboxes_normalized_initial.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])
            class_labels.append(class_id)
        
        print(f"DEBUG: Bboxes normalizadas (iniciales): {len(bboxes_normalized_initial)}")
        if bboxes_normalized_initial:
            print(f"DEBUG: Primer bbox normalizada (inicial): {bboxes_normalized_initial[0]}, clase: {class_labels[0]}")

        if self.transform:
            # Albumentations ahora recibe coordenadas normalizadas y las transformará.
            # Se espera que devuelva coordenadas normalizadas también.
            transformed = self.transform(image=image, bboxes=bboxes_normalized_initial, class_labels=class_labels)
            image = transformed['image']
            bboxes_transformed_raw = transformed['bboxes'] # Bboxes después de Albumentations (deberían estar normalizadas)
            class_labels = transformed['class_labels'] # Las etiquetas de clase se mantienen
            
        print(f"DEBUG: Bboxes después de Albumentations (raw, deberían estar normalizadas): {len(bboxes_transformed_raw)}")
        if bboxes_transformed_raw:
            print(f"DEBUG: Primer bbox después de Albumentations (raw, deberían estar normalizadas): {bboxes_transformed_raw[0]}")

        # --- ELIMINAR PASO DE RE-NORMALIZACIÓN HEURÍSTICA ---
        # Si Albumentations funciona como se espera con format='albumentations',
        # este paso ya no es necesario.
        bboxes = bboxes_transformed_raw # Usar las bboxes directamente de Albumentations
        
        print(f"DEBUG: Bboxes finales antes de YOLO format: {len(bboxes)}")
        if bboxes:
            print(f"DEBUG: Primer bbox final antes de YOLO format: {bboxes[0]}")
        # --- FIN ELIMINAR PASO ---


        # Si ToTensorV2 ya se aplicó, la imagen es un tensor. Si no, convertirla.
        if not isinstance(image, torch.Tensor):
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        yolo_bboxes = []
        for i, bbox in enumerate(bboxes):
            x_min, y_min, x_max, y_max = bbox
            
            # Asegurar que las coordenadas estén dentro de [0, 1]
            x_min = max(0.0, min(1.0, x_min))
            y_min = max(0.0, min(1.0, y_min))
            x_max = max(0.0, min(1.0, x_max))
            y_max = max(0.0, min(1.0, y_max))

            center_x = (x_min + x_max) / 2
            width = x_max - x_min
            center_y = (y_min + y_max) / 2
            height = y_max - y_min
            
            # Este filtrado ya estaba, pero el error ocurría antes
            if width <= 0 or height <= 0:
                print(f"DEBUG: Bbox filtrada por width/height <= 0: {bbox}")
                continue # Saltar esta bbox inválida después de transformación

            yolo_bboxes.append([class_labels[i], center_x, center_y, width, height])
            
        print(f"DEBUG: Bboxes finales en formato YOLO: {len(yolo_bboxes)}")
        if yolo_bboxes:
            print(f"DEBUG: Primer bbox YOLO: {yolo_bboxes[0]}")

        if len(yolo_bboxes) == 0:
            # Devuelve un tensor vacío si no hay bboxes válidas
            yolo_bboxes = torch.zeros((0, 5), dtype=torch.float32)
        else:
            yolo_bboxes = torch.tensor(yolo_bboxes, dtype=torch.float32)
        
        return image, yolo_bboxes

# PASO 2: Definición de las transformaciones de Albumentations
# Define el tamaño de entrada de tu modelo YOLOv3 (416x416)

YOLO_INPUT_SIZE = (416, 416) 

# --- TRANSFORMACIONES DE ENTRENAMIENTO CON AUMENTACIÓN DE COLOR/APARIENCIA ---
train_transforms = A.Compose([
    # Redimensionamiento y Relleno
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    
    # Transformaciones de Color y Apariencia
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.1), # Asegúrate de que blur_limit es impar y no demasiado grande
    
    # Normalización y Conversión a Tensor
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2(), 
], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])) # El formato 'albumentations' espera y devuelve normalizado [0,1]

# Las transformaciones de validación/prueba se mantienen minimalistas
val_test_transforms = A.Compose([
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])) 

# PASO 3: Definición de la función Collate_fn para el DataLoader
def collate_fn(batch):
    images = []
    bboxes = []
    for img, bbox_target in batch:
        images.append(img)
        bboxes.append(bbox_target) 
    images = torch.stack(images, 0)
    return images, bboxes

# PASO 4: Definición de la Lógica de División del Dataset y Creación de DataLoaders
# Modificada para que visualice las imágenes con las bounding boxes

if __name__ == '__main__':
    # RUTAS A LOS DATOS
    DATA_ROOT = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset'
    CSV_FILE = os.path.join(DATA_ROOT, 'annotations.csv') 
        
    # Parámetros de la división
    TEST_SPLIT_RATIO = 0.15    
    VAL_SPLIT_RATIO = 0.15     
    RANDOM_SEED = 42           

    BATCH_SIZE = 8
    NUM_WORKERS = 0 # Deja en 0 para depuración, luego puedes aumentarlo a 4-8

    # --- Cargar todas las anotaciones y obtener nombres de archivo únicos ---
    print(f"Cargando todas las anotaciones desde: {CSV_FILE}")
    full_df = pd.read_csv(CSV_FILE)
    
    # Obtener la lista de nombres de archivo únicos presentes en el CSV
    all_image_filenames = full_df['filename'].unique().tolist()
    print(f"Total de {len(all_image_filenames)} imágenes únicas encontradas en el CSV.")

    # --- Dividir los nombres de archivo en entrenamiento y test ---
    train_val_filenames, test_filenames = train_test_split(
        all_image_filenames, 
        test_size=TEST_SPLIT_RATIO, 
        random_state=RANDOM_SEED
    )
    
    train_filenames, val_filenames = train_test_split(
        train_val_filenames, 
        test_size=VAL_SPLIT_RATIO / (1 - TEST_SPLIT_RATIO), 
        random_state=RANDOM_SEED
    )

    print(f"Imágenes para entrenamiento: {len(train_filenames)}")
    print(f"Imágenes para validación: {len(val_filenames)}")
    print(f"Imágenes para prueba: {len(test_filenames)}")

    # --- Crear DataFrames de anotaciones para cada split ---
    train_df = full_df[full_df['filename'].isin(train_filenames)].copy()
    val_df = full_df[full_df['filename'].isin(val_filenames)].copy()
    test_df = full_df[full_df['filename'].isin(test_filenames)].copy()

    # --- Crear instancias del Dataset y DataLoader para cada split ---
    train_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=train_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=train_transforms
    )
    val_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=val_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms 
    )
    test_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=test_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms 
    )

    train_dataloader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )

    print("\nDataset y DataLoaders de entrenamiento, validación y prueba configurados exitosamente.")

    # --- Verificación de la carga de un lote de entrenamiento ---
    print("\nVerificando la carga de un lote de entrenamiento...")
    MAX_BATCHES_TO_CHECK = 10 
    found_image_with_boxes = False

    for batch_idx, (images, targets) in enumerate(train_dataloader):
        print(f"Tamaño del lote {batch_idx+1}: Imágenes: {images.shape}, Targets: {len(targets)}")
        
        # Buscar una imagen con cajas en el lote actual
        for img_idx in range(len(targets)):
            if targets[img_idx].numel() > 0: 
                print(f"--- Encontrada imagen con {targets[img_idx].shape[0]} cajas en el lote {batch_idx+1}, imagen {img_idx+1} ---")
                print(f"Ejemplo de target para esta imagen (clase, cx, cy, w, h normalizados):")
                print(targets[img_idx][0])
                
                # --- Lógica de visualización ---
                mean = torch.tensor((0.485, 0.456, 0.406)).view(3, 1, 1).to(images[img_idx].device)
                std = torch.tensor((0.229, 0.224, 0.225)).view(3, 1, 1).to(images[img_idx].device)
                
                img_display_rgb = (images[img_idx] * std + mean) * 255
                img_display_rgb = img_display_rgb.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
                img_display_bgr = cv2.cvtColor(img_display_rgb, cv2.COLOR_RGB2BGR)
                
                img_h, img_w = img_display_bgr.shape[:2]
                
                CLASS_ID_TO_NAME_MAP = {0: 'RBC', 1: 'WBC', 2: 'Platelets'}
                CLASS_COLORS_MAP = {0: (0, 0, 255), 1: (0, 255, 0), 2: (255, 0, 0)} # BGR

                print("\nVisualizando la imagen con GT Boxes (presiona cualquier tecla para cerrar)...")
                for bbox_yolo in targets[img_idx].tolist():
                    class_id, cx, cy, w, h = bbox_yolo
                    
                    x_min_norm = cx - w/2
                    y_min_norm = cy - h/2
                    x_max_norm = cx + w/2
                    y_max_norm = cy + h/2

                    x_min_px = int(x_min_norm * img_w)
                    y_min_px = int(y_min_norm * img_h)
                    x_max_px = int(x_max_norm * img_w)
                    y_max_px = int(y_max_norm * img_h)

                    color = CLASS_COLORS_MAP.get(int(class_id), (255, 255, 255)) 
                    cv2.rectangle(img_display_bgr, (x_min_px, y_min_px), (x_max_px, y_max_px), color, 2)

                    label_text = f"{CLASS_ID_TO_NAME_MAP.get(int(class_id), 'Unknown')}"
                    text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                    text_x = x_min_px
                    text_y = y_min_px - 5 if y_min_px - 5 > 5 else y_min_px + text_size[1] + 5
                    
                    cv2.rectangle(img_display_bgr, (text_x, text_y - text_size[1] - 5), 
                                (text_x + text_size[0] + 5, text_y + 5), color, -1)
                    cv2.putText(img_display_bgr, label_text, (text_x, text_y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

                cv2.imshow("Imagen con GT Boxes", img_display_bgr)
                cv2.waitKey(0) 
                cv2.destroyAllWindows()
                
                found_image_with_boxes = True
                break 
        
        if found_image_with_boxes or batch_idx + 1 >= MAX_BATCHES_TO_CHECK:
            break 

    if not found_image_with_boxes:
        print(f"\nNo se encontró ninguna imagen con bounding boxes en los primeros {MAX_BATCHES_TO_CHECK} lotes.")
        print("Esto podría deberse a que todas las imágenes mostradas no tenían bboxes o fueron filtradas.")
        print("Considera revisar:")
        print("1. El contenido de 'annotations.csv' para asegurar que hay bboxes válidas.")
        print("2. Los filtros en BloodCellDataset (xmin >= xmax, etc.).")
        print("3. Los parámetros de bbox en Albumentations (min_area, min_visibility).")
        print("4. Si RandomCrop está eliminando demasiadas bboxes si son pequeñas o están en los bordes.")

Cargando todas las anotaciones desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset\annotations.csv
Total de 364 imágenes únicas encontradas en el CSV.
Imágenes para entrenamiento: 254
Imágenes para validación: 55
Imágenes para prueba: 55
Dataset inicializado con 254 imágenes.
Dataset inicializado con 55 imágenes.
Dataset inicializado con 55 imágenes.

Dataset y DataLoaders de entrenamiento, validación y prueba configurados exitosamente.

Verificando la carga de un lote de entrenamiento...
DEBUG: Imagen original (H, W): (480, 640)
DEBUG: __getitem__ para BloodImage_00201.jpg. Bboxes iniciales (píxeles): 12
DEBUG: Primer bbox pixel: [302, 9, 406, 100, 0]
DEBUG: Bboxes normalizadas (iniciales): 12
DEBUG: Primer bbox normalizada (inicial): [0.471875, 0.01875, 0.634375, 0.20833333333333334], clase: 0
DEBUG: Bboxes después de Albumentations (raw, deberían estar normalizadas): 12
DEBUG: Primer bbox después de Albumentations (raw, deberían estar normalizadas): (0.47187499